## Scenario 1: A single data scientist participating in an ML competition
MLflow setup:

- Tracking server: no
- Backend store: local filesystem
- Artifacts store: local filesystem

The experiments can be explored locally by launching the MLflow UI.



In [1]:
import mlflow
from mlflow.tracking import MlflowClient

In [3]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'file:///Users/luisvaras/code/zoomcamp/mlops-zoomcamp/02-experiment_tracking/running-mlflow-examples/mlruns'


In [4]:
client = MlflowClient()

experiments = client.search_experiments()
print(f"experiments: {experiments}")

experiments: [<Experiment: artifact_location='file:///Users/luisvaras/code/zoomcamp/mlops-zoomcamp/02-experiment_tracking/running-mlflow-examples/mlruns/0', creation_time=1737770089618, experiment_id='0', last_update_time=1737770089618, lifecycle_stage='active', name='Default', tags={}>]


### Creating an experiment and logging a new run


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

experiment_name = "my-experiment-1"

experiment = client.get_experiment_by_name(experiment_name)

if experiment is None:
    experiment_id = client.create_experiment(experiment_name)
else:
    experiment_id = experiment.experiment_id

mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='file:///Users/luisvaras/code/zoomcamp/mlops-zoomcamp/02-experiment_tracking/running-mlflow-examples/mlruns/573603214355802727', creation_time=1737770112783, experiment_id='573603214355802727', last_update_time=1737770112783, lifecycle_stage='active', name='my-experiment-1', tags={}>

In [9]:
with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/01/24 22:57:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 'file:///Users/luisvaras/code/zoomcamp/mlops-zoomcamp/02-experiment_tracking/running-mlflow-examples/mlruns/573603214355802727/2e113098a70c436a808f4bfcfede0838/artifacts'


In [10]:
# list experiments
experiments = client.search_experiments()
print(f"experiments: {experiments}")

experiments: [<Experiment: artifact_location='file:///Users/luisvaras/code/zoomcamp/mlops-zoomcamp/02-experiment_tracking/running-mlflow-examples/mlruns/573603214355802727', creation_time=1737770112783, experiment_id='573603214355802727', last_update_time=1737770112783, lifecycle_stage='active', name='my-experiment-1', tags={}>, <Experiment: artifact_location='file:///Users/luisvaras/code/zoomcamp/mlops-zoomcamp/02-experiment_tracking/running-mlflow-examples/mlruns/0', creation_time=1737770089618, experiment_id='0', last_update_time=1737770089618, lifecycle_stage='active', name='Default', tags={}>]


### Interacting with the model registry

In [11]:
from mlflow.exceptions import MlflowException

client = MlflowClient()
try:
    client.search_registered_models()
except MlflowException as e:
    print("It's not possible to access the model regitry")